In [47]:
from playwright.async_api import async_playwright
import asyncio

from datetime import datetime, timedelta

### Helper function for get real publish time. 
### Convert Bangla number into English Number

In [50]:
# Mapping Bengali digits to English
bn_digit_map = {
    '০': '0', '১': '1', '২': '2', '৩': '3', '৪': '4',
    '৫': '5', '৬': '6', '৭': '7', '৮': '8', '৯': '9'
}

def bn_to_en_digits(bn_str):
    return ''.join([bn_digit_map.get(ch, ch) for ch in bn_str])
    
###########################################################################
# Function for convert bangla time english and return real published time    
###########################################################################

def parse_bengali_relative_time(bn_time_str):
    if "No time found" in bn_time_str:
        return "No time found"
        
    now = datetime.now()
    
    bn_time_str = bn_time_str.strip()
    
    # Extract number
    num_str = ''.join([bn_digit_map.get(ch, ch) for ch in bn_time_str if ch in bn_digit_map or ch.isdigit()])
    num = int(num_str) if num_str else 0

    if 'সেকেন্ড' in bn_time_str:
        delta = timedelta(seconds=num)
    elif 'মিনিট' in bn_time_str:
        delta = timedelta(minutes=num)
    elif 'ঘণ্টা' in bn_time_str:
        delta = timedelta(hours=num)
    elif 'দিন' in bn_time_str:
        delta = timedelta(days=num)
    else:
        delta = timedelta()  # fallback

    return now - delta

In [52]:
async def fetch_with_playwright():
    news_list = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        
        page = await browser.new_page()
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
        })
        await page.goto("https://www.prothomalo.com/collection/latest", timeout=60000)
        await page.wait_for_selector("#container", timeout=15000)
        #Wait for a card to appear inside container
        await page.wait_for_selector("#container div.news_with_item", timeout=10000)
        elements = await page.locator("h3.headline-title").all()
        for el in elements:
            link_el  =  el.locator("a.title-link")
            href = await link_el.get_attribute("href")
            title = (await el.inner_text()).strip()
            # Get the nearest ancestor or surrounding time tag with class 'published-time'
            time_locator = el.locator("xpath=following::time[contains(@class, 'published-time')]")
        
            # Check if it exists
            if await time_locator.count() > 0:
                published_time = await time_locator.first.inner_text()
            else:
                published_time = "No time found"
            print("published_time", published_time)
            news_list.append((title, href, published_time))
            
        await browser.close()
    return news_list
        #for i in range(articles.count())
     

In [55]:
news_list = await fetch_with_playwright() 
#print(news_list)

published_time ৫ মিনিট আগে
published_time ৫ মিনিট আগে
published_time ৬ মিনিট আগে
published_time ৬ মিনিট আগে
published_time ৯ মিনিট আগে
published_time ১৩ মিনিট আগে
published_time ১৩ মিনিট আগে
published_time ১৩ মিনিট আগে
published_time ১৩ মিনিট আগে
published_time ১৩ মিনিট আগে
published_time ১৩ মিনিট আগে
published_time ১৭ মিনিট আগে
published_time ১৮ মিনিট আগে
published_time ১৮ মিনিট আগে
published_time ২১ মিনিট আগে
published_time ২১ মিনিট আগে
published_time ২২ মিনিট আগে
published_time ২৩ মিনিট আগে
published_time ২৫ মিনিট আগে
published_time ২৮ মিনিট আগে
published_time ২৯ মিনিট আগে
published_time ৩০ মিনিট আগে
published_time ৩৬ মিনিট আগে
published_time ৩৭ মিনিট আগে
published_time ৫৬ মিনিট আগে


In [63]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/") # need to on env. for local can't do it right now.
db = client["bd-newspaper-collection"]

collection = db["articles"]

for title,url, published_time in news_list:
    published_time = parse_bengali_relative_time(published_time)
    print("published_time",published_time)
    if not collection.find_one({"url": url}):
        
        collection.insert_one({
            "title": title,
            "url": url,
            "published_time": published_time,
            "media_name": "prothomalo",
            "scraped_at": datetime.now()
        })
        print(f"Inserted: {title}")
    else:
        print(f"Already Exits {title}")


published_time 2025-04-09 18:10:01.271866
Already Exits সংস্কার নিয়ে মতামত দিল আরও দুই দল
published_time 2025-04-09 18:10:01.292378
Already Exits ব্যবসায়ী নেতার সাক্ষাৎকার ব্যবসাপ্রতিষ্ঠানে হামলা-লুটপাট সিলেটের চিরায়ত সম্প্রীতিবিরোধী কর্মকাণ্ড
published_time 2025-04-09 18:09:01.293377
Already Exits পাঠকের ছবি
published_time 2025-04-09 18:09:01.294378
Already Exits মাত্র ৮ ঘণ্টার জন্য ঢাকায়—কী হয়েছিল শাবনূরের
published_time 2025-04-09 18:06:01.295378
Already Exits ম্যারাডোনাকে বাসায় নিয়ে যাওয়াটাই ভুল হয়েছিল, বললেন চিকিৎসক
published_time 2025-04-09 18:02:01.297376
Already Exits বাংলাদেশের ট্রান্সশিপমেন্ট সুবিধা বাতিল করল ভারত
published_time 2025-04-09 18:02:01.298376
Already Exits সরাসরি যুক্তরাষ্ট্রের ১০৪ শতাংশের জবাবে চীনের ৮৪ শতাংশ শুল্ক, তেলের দাম ৬০ ডলার
published_time 2025-04-09 18:02:01.299378
Already Exits কাঠগড়ায় মেজাজ হারিয়ে চিৎকার-চেঁচামেচি হাজি সেলিমের, অন্যরা হতবাক
published_time 2025-04-09 18:02:01.300378
Already Exits এনসিপির সঙ্গে হেফাজতের বৈঠক, আওয়ামী লীগের বিরুদ্ধে ঐকমত

In [59]:
len(news_list)

25